In [1]:
# Importing the libraries
import numpy as np
import pandas as pd
import re
import pickle 
import nltk
from nltk.corpus import stopwords
from sklearn.datasets import load_files
nltk.download('stopwords')

# stopwordsleri sil
from nltk.corpus import stopwords
stop = stopwords.words('Turkish')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/uzaycetin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Turkish Stopwords
with open('17bintweet/turkce-stop-words.txt') as file:  
    stw = file.read() 
stw = stw.split()
stw = [s.lower() for s in stw] 
stop += stw
#stop

In [3]:
from snowballstemmer import stemmer
kokbul1 = stemmer('turkish')

In [4]:
def preprocessing(text):
    text = text.lower()
    # get rid of non-alphanumerical characters
    text = re.sub(r'\W', ' ', text) 
    # get rid of spaces
    text = re.sub(r'\s+', ' ', text) 
    # Correct mistakes 
    # and do the stemming
    return " ".join([word for word in kokbul1.stemWords(text.split()) if word not in stop])

In [5]:
test_data = pd.read_excel('17bintweet/test_tweets.xlsx', header=None)
test_data.head(10)

,0,1
0,Turkcell'e kızgınım. Ve bu kızgınlık sanırım a...,olumsuz
1,turkcell kadar şerefsiz misiniz ya,olumsuz
2,Burdan Turkcell'e sesleniyorum o 3 tl haram olsun,olumsuz
3,Hayatımda turkcell kadar kazık 1 operatör görm...,olumsuz
4,Turkcell gözümde son demlerini yaşıyor hattı d...,olumsuz
5,Sonra turkcell olduğunu görüp sövüyorum,olumsuz
6,Turkcell smsle internetimi göndermedi abv turk...,olumsuz
7,Turkcell sen ne kadar kaşarsın,olumsuz
8,şu zamana kadar gördüğüm en kazık operatör tur...,olumsuz
9,"TTNet Pişmanlıktır , Digitürk Pişmanlıktır, Tu...",olumsuz


In [6]:
train_data = pd.read_excel('17bintweet/train_tweets.xlsx', header=None)
train_data.head(10)

,0,1
0,Ulan Wifi'ye bağlıyım ben. Ona bağlıyken Turkc...,olumsuz
1,20 dk 1 GB internet 500 mb sadece kaşar turkce...,olumsuz
2,Ayrıca turkcell superonline reklamı kadar da k...,olumsuz
3,Turkcell çok pahalı ya,olumsuz
4,Turkcell Kaş'ta internetin cekmiyor,olumsuz
5,Turkcell'in Allah belası versin demek isterdim...,olumsuz
6,Canın cehenneme turkcell,olumsuz
7,Turkcell yönetimini eline geçiren AKP hükümeti...,olumsuz
8,Turkcell şerefsizdir aksini iddia eden turkcel...,olumsuz
9,Turkcell'den elimi ayağımı çektiğim iyi oldu,olumsuz


In [39]:
data = pd.concat([train_data, test_data])
data.head()

,0,1
0,Ulan Wifi'ye bağlıyım ben. Ona bağlıyken Turkc...,olumsuz
1,20 dk 1 GB internet 500 mb sadece kaşar turkce...,olumsuz
2,Ayrıca turkcell superonline reklamı kadar da k...,olumsuz
3,Turkcell çok pahalı ya,olumsuz
4,Turkcell Kaş'ta internetin cekmiyor,olumsuz


In [7]:
text = train_data.iloc[0,0]
text = preprocessing(text)
text

'ula wif bağlı bağlı turkcell internet paket bit di mesaj atabilir ba ödeyel'

In [8]:
len(train_data)

13832

In [40]:
# Creating the corpus
corpus = []
for i in range(len(data)):
    corpus.append(preprocessing(data.iloc[i,0]))  

In [45]:
# Creating the Tf-Idf model directly
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [52]:
X_train =  vectorizer.transform(corpus[:len(train_data)]).toarray()
X_test =  vectorizer.transform(corpus[len(train_data):]).toarray()

In [62]:
sentiment = {'olumsuz':0, 'notr':1, 'olumlu':2}
inv_sentiment = {v:k for k, v in sentiment.items()}
inv_sentiment

{0: 'olumsuz', 1: 'notr', 2: 'olumlu'}

In [53]:
len(vectorizer.vocabulary_)

25967

In [54]:
y_train = np.array([sentiment[s] for s in train_data.iloc[:,1]])
y_test = np.array([sentiment[s] for s in test_data.iloc[:,1]])
y_train

array([0, 0, 0, ..., 0, 1, 2])

In [55]:
# Training the classifier
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [56]:
# Testing model performance
y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
cm

array([[1126,  169,   82],
       [ 316,  710,  138],
       [ 222,  191,  503]])

In [57]:
accuracy_score(y_test, y_pred)

0.6765982065374603

In [64]:
raw_sample = ["Kampanya berbat.  kötü ya!", "güzel olmuş", "kötü değil", "müthiş harika sevdim"]
# Creating the corpus
test_sample = []
for i in range(len(raw_sample)):
    test_sample.append(preprocessing(raw_sample[i]))  

sample = vectorizer.transform(test_sample).toarray()
result = classifier.predict(sample)
result

array([0, 2, 0, 2])

In [65]:
[inv_sentiment[r] for r in result]

['olumsuz', 'olumlu', 'olumsuz', 'olumlu']